In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eclipse-preprocessed5new/eclipse_preprocessed_os.csv


In [2]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/tools/tokenization.py

In [3]:
!pip install sentencepiece

In [4]:
!pip install tensorflow_hub

In [5]:
!pip install torch

In [6]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")
nltk.download("words")
from nltk.corpus import stopwords
from nltk.corpus import words
# words = set(nltk.corpus.words.words())

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [7]:
import pandas as pd
import numpy as np
from random import seed
from random import randint
import random
import string
import re
import matplotlib.pyplot as plt
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
from tensorflow.keras.layers import Dense, Input, Reshape, Conv1D, Conv2D, BatchNormalization, MaxPooling1D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tokenization
from sklearn.model_selection import StratifiedKFold

In [9]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

2022-03-20 12:34:43.016291: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-20 12:34:43.063740: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-20 12:34:43.147445: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-20 12:34:43.148475: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [10]:
seed(1)
stop_words = stopwords.words('english')
words=words.words()

In [11]:
# train_indices = []
# test_indices = []

# with open('../input/eclipse-train-test5/train5_dt5.txt') as f:
#     train_indices = f.readlines()

# with open('../input/eclipse-train-test5/test5_dt5.txt') as f:
#     test_indices = f.readlines()

In [12]:
# train_indices = [int(tr.split('\n')[0]) for tr in train_indices]

In [13]:
# len(train_indices)

In [14]:
# test_indices  = [int(te.split('\n')[0]) for te in test_indices]

In [15]:
# len(test_indices)

In [16]:
dataset = pd.read_csv('../input/eclipse-preprocessed5new/eclipse_preprocessed_os.csv')
dataset

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,bug_id,description1,short_desc1,dup_id,resolution,product,component,description2,short_desc2,duplicate
0,59556,62303,There should be some global dialog or preferen...,[Keybindings] Provide a list of all keybindings,37439,DUPLICATE,Platform,UI,35722 it would be very useful it if were po...,35722 [KeyBindings] preference page: add a ...,1
1,25380,26824,"When class X has a private field F, and code i...",New auto-suggestion; add 'generate getXxx' whe...,28068,DUPLICATE,JDT,UI,26619 I have two classes:\nA - with private...,"26619 [quick fix] Greate getter for ""Field ...",1
2,84739,89233,Need to complete a WSDL model core API accordi...,Need to define WSDL Core API,89291,DUPLICATE,WTP Webservices,wst.wsdl,84804 Need to complete a WSDL model core AP...,84804 Need to define WSDL Core API,1
3,145562,155467,Build: RSA-M20060802-0800\n\nSteps:\n1) Wind...,No error message displays when choose an inval...,155442,DUPLICATE,EPF,Tool,145537 Build: RSA-M20060802-0800\n\nSteps:\...,145537 No error message displays when choos...,1
4,186715,201768,Build ID: I20070625-1500\n\nSteps To Reproduce...,Should be able to middle click on editor/ view...,156792,DUPLICATE,Platform,UI,146782 It would be ui-friendly if the tabs ...,146782 [EditorMgmt] Close a Tab with the mi...,1
...,...,...,...,...,...,...,...,...,...,...,...
241158,189743,205251,Build ID: M20060921-0945\n\nSteps To Reproduce...,CVS username disallows full kerberos names,[],FIXED,Platform,CVS,The current TPTP source locator only searches ...,Need non-Java centric extensible source locator,0
241159,16340,17624,F1 Linux Motif \n\nMnemonics do not work in ec...,Mnemonics do not work,[],NDUPLICATE,Platform,SWT,Build Identifier: 4.7.0_201004221916\n\nPrevio...,Execution Statistics view is populated even wh...,0
241160,285393,320100,1. Open a fresh e4 workbench\n2. Leave the Pac...,Package Explorer unexpectedly added to Debug p...,[],FIXED,e4,UI,"""Content Assist"" did not complete normally. P...","""Content Assist"" did not complete normally.",0
241161,237512,261239,Created attachment 122704\nImage of the proble...,SWT Styled Text corrupts screen in Arabic on M...,[],INVALID,Platform,SWT,When comparing files which contain CVS keyword...,Option to ignore CVS keywords,0


In [17]:
dataset.dropna(subset = ["description1"], inplace=True)
dataset.dropna(subset = ["description2"], inplace=True)

In [18]:
# dataset.loc[dataset['bug_id'] == 15957]

In [19]:
for index, row in dataset.iterrows():
    doc1 = 'Product:' + row['product'] + '.Component:' + row['component'] + '.Description:' + row['description1'] + '.Summary:' + row['short_desc1']
#     print(row['bug_id'])
#     print(row['description2'])
    doc2 = 'Product:' + row['product'] + '.Component:' + row['component'] + '.Description:' + row['description2'] + '.Summary:' + row['short_desc2']
    dataset.loc[index, 'doc1'] = doc1
    dataset.loc[index, 'doc2'] = doc2

In [20]:
for index, row in dataset.iterrows():
    desc = row['doc1'] + row['doc2']
    dataset.loc[index, 'description'] = desc

In [21]:
def reset_column_names():
  dataset.drop('description', axis=1, inplace=True)
#   dataset.drop('short_desc1', axis=1, inplace=True)
#   dataset.drop('description2', axis=1, inplace=True)
#   dataset.drop('short_desc2', axis=1, inplace=True)

  dataset.rename(columns={'description_clean':'description'}, inplace=True) #,'short_desc1_clean':'short_desc1','description2_clean':'description2','short_desc2_clean':'short_desc2'

In [22]:
def preprocess_text_content(text):
    text = text.lower()
#     cleaned_text = re.sub(r"""
#                [,.;:@#?!&$*+-=_%<>\/\[\]\(\)\{\}\"\'\n]+  # Accept one or more copies of punctuation
#                \ *           # plus zero or more copies of a space,
#                """,
#                " ",          # and replace it with a single space
#                text, flags=re.VERBOSE)
#     cleaned_text = re.sub(r"https?:\/\/[A-Za-z0-9.\/?&#+*+-=_%]+", " ",text) #replace urls
    cleaned_text = re.sub(r" +", " ", text) #remove extra white spaces
#     cleaned_text = re.sub(r"[d|D]escription", "", cleaned_text)
    
    return cleaned_text

In [23]:
dataset['description_clean']= dataset['description'].apply(lambda x:preprocess_text_content(x))
# dataset['short_desc1_clean']= dataset['short_desc1'].apply(lambda x:preprocess_text(x))
# dataset['description2_clean']= dataset['description2'].apply(lambda x:preprocess_text(x))
# dataset['short_desc2_clean']= dataset['short_desc2'].apply(lambda x:preprocess_text(x))
dataset.head()

,Unnamed: 0,bug_id,description1,short_desc1,dup_id,resolution,product,component,description2,short_desc2,duplicate,doc1,doc2,description,description_clean
0,59556,62303,There should be some global dialog or preferen...,[Keybindings] Provide a list of all keybindings,37439,DUPLICATE,Platform,UI,35722 it would be very useful it if were po...,35722 [KeyBindings] preference page: add a ...,1,Product:Platform.Component:UI.Description:Ther...,Product:Platform.Component:UI.Description:3572...,Product:Platform.Component:UI.Description:Ther...,product:platform.component:ui.description:ther...
1,25380,26824,"When class X has a private field F, and code i...",New auto-suggestion; add 'generate getXxx' whe...,28068,DUPLICATE,JDT,UI,26619 I have two classes:\nA - with private...,"26619 [quick fix] Greate getter for ""Field ...",1,Product:JDT.Component:UI.Description:When clas...,Product:JDT.Component:UI.Description:26619 ...,Product:JDT.Component:UI.Description:When clas...,product:jdt.component:ui.description:when clas...
2,84739,89233,Need to complete a WSDL model core API accordi...,Need to define WSDL Core API,89291,DUPLICATE,WTP Webservices,wst.wsdl,84804 Need to complete a WSDL model core AP...,84804 Need to define WSDL Core API,1,Product:WTP Webservices.Component:wst.wsdl.Des...,Product:WTP Webservices.Component:wst.wsdl.Des...,Product:WTP Webservices.Component:wst.wsdl.Des...,product:wtp webservices.component:wst.wsdl.des...
3,145562,155467,Build: RSA-M20060802-0800\n\nSteps:\n1) Wind...,No error message displays when choose an inval...,155442,DUPLICATE,EPF,Tool,145537 Build: RSA-M20060802-0800\n\nSteps:\...,145537 No error message displays when choos...,1,Product:EPF.Component:Tool.Description:Build: ...,Product:EPF.Component:Tool.Description:145537 ...,Product:EPF.Component:Tool.Description:Build: ...,product:epf.component:tool.description:build: ...
4,186715,201768,Build ID: I20070625-1500\n\nSteps To Reproduce...,Should be able to middle click on editor/ view...,156792,DUPLICATE,Platform,UI,146782 It would be ui-friendly if the tabs ...,146782 [EditorMgmt] Close a Tab with the mi...,1,Product:Platform.Component:UI.Description:Buil...,Product:Platform.Component:UI.Description:1467...,Product:Platform.Component:UI.Description:Buil...,product:platform.component:ui.description:buil...


In [24]:
reset_column_names()

In [25]:
# def preprocess_text(text,tokenizer):
# #     print(text)
#     #tokenize the text
#     text = tokenizer.tokenize(text)
#     #lowercase the text
#     text = [w.lower() for w in text]
#     #removal of stop word 
#     text = [w for w in text if w not in stop_words]
#     #removal of non-english words
#     text = [w for w in text if w in words or w.isalpha()]
    #stemming
#     text = [ ps.stem(w) for w in text]
    #remove extra white spaces
#     text = [ re.sub(r" +", " ", w) for w in text] 
#     cleaned_text = re.sub(r"""
#                [,.;:@#?!&$*+-=_%<>\/\[\]\(\)\{\}\"\'\n]+  # Accept one or more copies of punctuation
#                \ *           # plus zero or more copies of a space,
#                """,
#                " ",          # and replace it with a single space
#                text, flags=re.VERBOSE)
#     cleaned_text = re.sub(r"https?:\/\/[A-Za-z0-9.\/?&#+*+-=_%]+", " ",cleaned_text) #replace urls
    
#     cleaned_text = re.sub(r"[d|D]escription", "", cleaned_text)
    
#     return text

In [26]:
max_len = 0
# lenArr = []
# outliers = []

# q25, q75 = np.percentile(lenArr, [25, 75])
# bin_width = 2 * (q75 - q25) * len(lenArr) ** (-1/3)
# bins = round((lenArr.max() - lenArr.min()) / bin_width)

### Add tokens to the data make it BERT compatible
def bert_encode(texts, tokenizer, max_len=512):
    print('encode')
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
#         text = preprocess_text(text,tokenizer)
        text = tokenizer.tokenize(text)
#         print(text)
        text = [w for w in text if w.lower() not in stop_words]
    
#         length = len(text)
#         if(length <1000):
#             lenArr.append(length)
#     #     print(length)
#             if(max_len < length):
#               max_len = length
#         else:
#             outliers.append(length)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
#     print(len(lenArr))
#     # lenArr=list(filter(lambda a: a != 3395, lenArr))
#     # print(lenArr)
#     print("avg", sum(lenArr)/len(lenArr))
#     print("outliers", outliers)
#     print("outliers", len(outliers))
#     plt.hist(lenArr, density=True, bins=30)
#     max_len

    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [27]:
def build_model(bert_layer, max_len=512):
    print('build model')
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    
#     embedding = bert_layer(
#         input_word_ids, token_type_ids=segment_ids, attention_mask=input_mask
#     )[0]

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :] 
#     print(clf_output)
   
    clf_output = Reshape((1024,1))(clf_output)
    print(clf_output)
    
    x = Conv1D(8, 3, activation='relu', padding='same')(clf_output)
    x = BatchNormalization()(x)
    x = MaxPooling1D()(x)
    x = Conv1D(32, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D()(x)
#     x = Flatten()(x)
#     out = Dense(1, activation='sigmoid')(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    
    return model

In [28]:
train, test = train_test_split(dataset, test_size=0.2)

In [29]:
FullTokenizer = tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1", trainable=True)

2022-03-20 13:56:10.529443: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-20 13:56:10.530303: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-20 13:56:10.530912: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-20 13:56:10.531801: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-20 13:56:10.532479: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [30]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
lower_cased = bert_layer.resolved_object.do_lower_case.numpy()

tokenizer = FullTokenizer(vocab_file, lower_cased)

In [31]:
# X = dataset['description']
# Y = np.array(dataset.duplicate.values, dtype='int')

In [32]:
train_input = bert_encode(train['description'], tokenizer, max_len=160)
train_labels = np.array(train.duplicate.values, dtype='int')

encode


In [33]:
model = build_model(bert_layer, max_len=160)
model.summary()

# Fit the model
model.fit(train_input,train_labels, epochs=5, batch_size=20)

build model
KerasTensor(type_spec=TensorSpec(shape=(None, 1024, 1), dtype=tf.float32, name=None), name='reshape/Reshape:0', description="created by layer 'reshape'")
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)   

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-03-20 14:29:30.155097: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5


ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:835 run_step  **
        outputs = model.train_step(data)
    /opt/conda/lib/python3.7/site-pack

In [ ]:
model.save("eclipse_bert_cnn")

In [ ]:
test_input = bert_encode(test['description'], tokenizer, max_len=160)
test_labels = np.array(test.duplicate.values, dtype='int')

In [ ]:
scores = model.evaluate(test_input, test_labels, verbose=0)
f1score = (2 * scores[2]*100 * scores[3]*100)/(scores[2]*100 + scores[3]*100)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
print("f1 score: ", (f1score))

In [ ]:
# train_input = bert_encode(dataset['description'], tokenizer, max_len=160)
# test_input = bert_encode(test['description'], tokenizer, max_len=160)
# train_labels = np.array(train.duplicate.values, dtype='int')
# test_labels = np.array(test.duplicate.values, dtype='int')

In [ ]:
# max_len = 0
# lenArr = []
# outliers = []

# for index, row in dataset.iterrows():
#     # print(train.loc[[index]]['description'])
#     length = max([len(s.split()) for s in dataset.loc[[index]]['description']])
#     if(length <1000):
#         lenArr.append(length)
# #     print(length)
#         if(max_len < length):
#           max_len = length
#     else:
#         outliers.append(length)

# # q25, q75 = np.percentile(lenArr, [25, 75])
# # bin_width = 2 * (q75 - q25) * len(lenArr) ** (-1/3)
# # bins = round((lenArr.max() - lenArr.min()) / bin_width)
# print(len(lenArr))
# # lenArr=list(filter(lambda a: a != 3395, lenArr))
# # print(lenArr)
# print("avg", sum(lenArr)/len(lenArr))
# print("outliers", outliers)
# print("outliers", len(outliers))
# plt.hist(lenArr, density=True, bins=30)
# max_len

In [ ]:
# model = build_model(bert_layer, max_len=160)
# model.summary()

In [ ]:
# train_history = model.fit(
#     train_input, train_labels,
#     validation_split=0.3,
#     epochs=5,
#     batch_size=50
# )

In [ ]:
# evaluation_results = model.evaluate(test_input,test_labels,return_dict=True)
# evaluation_results

In [ ]:
# X = dataset['description']
# Y = np.array(dataset.duplicate.values, dtype='int')

# kfold = StratifiedKFold()
# cvscores = []
# precision_scores = []
# recall_scores = []
# f1_scores = []
    
# for train, test in kfold.split(X, Y):
#     # Prepare data
#     train_input = bert_encode(X.iloc[train], tokenizer, max_len=160)
#     test_input = bert_encode(X.iloc[test], tokenizer, max_len=160)
#     train_labels = Y[train]
#     test_labels = Y[test]
    
#     model = build_model(bert_layer, max_len=160)
#     model.summary()

#     # Fit the model
#     model.fit(train_input,train_labels, epochs=2, batch_size=15)

#     # Evaluate the model
#     scores = model.evaluate(test_input, test_labels, verbose=0)
#     f1score = (2 * scores[2]*100 * scores[3]*100)/(scores[2]*100 + scores[3]*100)
#     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
#     print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
#     print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
#     print("f1 score: ", (f1score))
#     cvscores.append(scores[1] * 100)
#     precision_scores.append(scores[2] * 100)
#     recall_scores.append(scores[3] * 100)
#     f1_scores.append(f1score)
    

# print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
# print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision_scores), np.std(precision_scores)))
# print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall_scores), np.std(recall_scores)))
# print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1_scores), np.std(f1_scores)))